#關聯分析 (Associative Analysis)

<p align="center">
    <img src="https://i.ibb.co/g4nNMRL/topic5-1-supervised-vs-unsupervised-learning.png" width="60%"/>
</p>





<p align="right">
    <a href='https://kaumadiechamalka100.medium.com/apriori-algorithm-f7fb30793274'>
    source
    </a>
</p>


## 範例資料

In [ ]:
%%bash
cat >  example.csv << EOL
userID,transactionsID,items
1,1,Milk|Onion|Nutmeg|Kidney Beans|Eggs|Yogurt
2,2,Dill|Onion|Nutmeg|Eggs
1,3,Milk|Apple|Kidney Beans|Eggs
3,4,Milk|Unicorn|Corn|Kidney Beans|Yogurt
2,5,Corn|Onion|Kidney Beans|Ice cream|Eggs
3,6,Ice cream|Eggs|Yogurt
4,7,Corn|Kidney Beans|Ice cream|Eggs
1,8,Corn|Onion|Ice cream
EOL

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter("ignore") #"error", "ignore", "always", "default", "module"

df = pd.read_csv('example.csv')
df

,userID,transactionsID,items
0,1,1,Milk|Onion|Nutmeg|Kidney Beans|Eggs|Yogurt
1,2,2,Dill|Onion|Nutmeg|Eggs
2,1,3,Milk|Apple|Kidney Beans|Eggs
3,3,4,Milk|Unicorn|Corn|Kidney Beans|Yogurt
4,2,5,Corn|Onion|Kidney Beans|Ice cream|Eggs
5,3,6,Ice cream|Eggs|Yogurt
6,4,7,Corn|Kidney Beans|Ice cream|Eggs
7,1,8,Corn|Onion|Ice cream


## 資料準備
在資料上通常會整理成 Adjacency Matrix(ii)或是Adjacency List(iii) 的格式。
下面為了方便講述使用圖論的示意圖，在 association rule 的情況下不完全一樣。


<p align="center">
    <img src="https://i.ibb.co/w7cJgXd/topic5-1-Digraph-G-ii-Adjacency-Matrix-of-G-iii-Adjacency-List-of-G-Considering-an.jpg" width="70%"/>
</p>

<p align="right">
    <a href='https://www.researchgate.net/figure/Fig-3-i-Digraph-G-ii-Adjacency-Matrix-of-G-iii-Adjacency-List-of-G-Considering-an_fig1_272172339'>
    source
    </a>
</p>

下圖是在 association rule 時常看到的格式，是使用 Adjacency Matrix 的表示方式。

<p align="center">
    <img src="https://i.ibb.co/zxxMkRG/topic5-1-tran-item-matrix.png" width="80%"/>
</p>





<p align="right">
    <a href='https://www.kaggle.com/code/keitazoumana/comparative-analysis-between-apriori-and-fp-growth'>
    source
    </a>
</p>


## 定義目標
在進行 association rule 前你需要先定義你算頻率的一筆代表的是甚麼，你要 mining 甚麼的 pattern。以上面的範例來說:




### transactions - items
一筆是代表一個交易，找商品間的 pattern。


In [ ]:
df_tran = df[['transactionsID', 'items']]
df_tran

,transactionsID,items
0,1,Milk|Onion|Nutmeg|Kidney Beans|Eggs|Yogurt
1,2,Dill|Onion|Nutmeg|Eggs
2,3,Milk|Apple|Kidney Beans|Eggs
3,4,Milk|Unicorn|Corn|Kidney Beans|Yogurt
4,5,Corn|Onion|Kidney Beans|Ice cream|Eggs
5,6,Ice cream|Eggs|Yogurt
6,7,Corn|Kidney Beans|Ice cream|Eggs
7,8,Corn|Onion|Ice cream


In [ ]:
items_ls = df_tran['items'].str.split('|').tolist()
items_ls

[['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
 ['Dill', 'Onion', 'Nutmeg', 'Eggs'],
 ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
 ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
 ['Corn', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs'],
 ['Ice cream', 'Eggs', 'Yogurt'],
 ['Corn', 'Kidney Beans', 'Ice cream', 'Eggs'],
 ['Corn', 'Onion', 'Ice cream']]

使用 [mlxtend.preprocessing.transactionencoder](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.preprocessing/#transactionencoder)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(items_ls).transform(items_ls)
df_tran_matrix = pd.DataFrame(te_ary, columns=te.columns_)
df_tran_matrix.set_index(df_tran['transactionsID'], inplace=True)
df_tran_matrix

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
transactionsID,,,,,,,,,,,
1,False,False,False,True,False,True,True,True,True,False,True
2,False,False,True,True,False,False,False,True,True,False,False
3,True,False,False,True,False,True,True,False,False,False,False
4,False,True,False,False,False,True,True,False,False,True,True
5,False,True,False,True,True,True,False,False,True,False,False
6,False,False,False,True,True,False,False,False,False,False,True
7,False,True,False,True,True,True,False,False,False,False,False
8,False,True,False,False,True,False,False,False,True,False,False


### users - items
一筆代表一個 user，找商品間的 pattern。

In [ ]:
df_user = df[['userID', 'items']]
df_user

,userID,items
0,1,Milk|Onion|Nutmeg|Kidney Beans|Eggs|Yogurt
1,2,Dill|Onion|Nutmeg|Eggs
2,1,Milk|Apple|Kidney Beans|Eggs
3,3,Milk|Unicorn|Corn|Kidney Beans|Yogurt
4,2,Corn|Onion|Kidney Beans|Ice cream|Eggs
5,3,Ice cream|Eggs|Yogurt
6,4,Corn|Kidney Beans|Ice cream|Eggs
7,1,Corn|Onion|Ice cream


In [ ]:
# 移除重複 user
df_user['items_new'] = df_user.groupby('userID')['items'].transform(lambda x: '|'.join(x))
df_user.drop('items', axis=1, inplace=True)
df_user.drop_duplicates(inplace=True)
df_user

,userID,items_new
0,1,Milk|Onion|Nutmeg|Kidney Beans|Eggs|Yogurt|Mil...
1,2,Dill|Onion|Nutmeg|Eggs|Corn|Onion|Kidney Beans...
3,3,Milk|Unicorn|Corn|Kidney Beans|Yogurt|Ice crea...
6,4,Corn|Kidney Beans|Ice cream|Eggs


In [ ]:
# 移除重複 item
df_user['items_new'] = df_user['items_new'].apply(lambda x: list(set(x.split('|'))))
df_user

,userID,items_new
0,1,"[Corn, Kidney Beans, Eggs, Onion, Ice cream, Y..."
1,2,"[Corn, Kidney Beans, Eggs, Onion, Ice cream, N..."
3,3,"[Corn, Kidney Beans, Eggs, Unicorn, Ice cream,..."
6,4,"[Corn, Eggs, Kidney Beans, Ice cream]"


使用 [mlxtend.preprocessing.transactionencoder](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.preprocessing/#transactionencoder)

In [ ]:
# 轉成 list
items_ls = df_user['items_new'].tolist()
items_ls

[['Corn',
  'Kidney Beans',
  'Eggs',
  'Onion',
  'Ice cream',
  'Yogurt',
  'Milk',
  'Nutmeg',
  'Apple'],
 ['Corn', 'Kidney Beans', 'Eggs', 'Onion', 'Ice cream', 'Nutmeg', 'Dill'],
 ['Corn', 'Kidney Beans', 'Eggs', 'Unicorn', 'Ice cream', 'Yogurt', 'Milk'],
 ['Corn', 'Eggs', 'Kidney Beans', 'Ice cream']]

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(items_ls).transform(items_ls)
df_user_matrix = pd.DataFrame(te_ary, columns=te.columns_)
df_user_matrix.set_index(df_user['userID'], inplace=True)
df_user_matrix

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
userID,,,,,,,,,,,
1,True,True,False,True,True,True,True,True,True,False,True
2,False,True,True,True,True,True,False,True,True,False,False
3,False,True,False,True,True,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,False,False,False


### items - user
一筆代表一個商品，找 user 間的 pattern。

In [ ]:
df_user

,userID,items_new
0,1,"[Corn, Kidney Beans, Eggs, Onion, Ice cream, Y..."
1,2,"[Corn, Kidney Beans, Eggs, Onion, Ice cream, N..."
3,3,"[Corn, Kidney Beans, Eggs, Unicorn, Ice cream,..."
6,4,"[Corn, Eggs, Kidney Beans, Ice cream]"


In [ ]:
# user - 商品清單
all_items = [(uid, i) for uid, items in df_user[['userID', 'items_new']].values for i in items]
df_items = pd.DataFrame(all_items, columns=['userID', 'items'])
df_items

,userID,items
0,1,Corn
1,1,Kidney Beans
2,1,Eggs
3,1,Onion
4,1,Ice cream
5,1,Yogurt
6,1,Milk
7,1,Nutmeg
8,1,Apple
9,2,Corn


In [ ]:
df_items['buy'] = 1
df_item_matrix = df_items.pivot(index='items', columns='userID', values='buy')
df_item_matrix.fillna(0, inplace=True)
df_item_matrix = df_item_matrix.astype(bool)
df_item_matrix

userID,1,2,3,4
items,,,,
Apple,True,False,False,False
Corn,True,True,True,True
Dill,False,True,False,False
Eggs,True,True,True,True
Ice cream,True,True,True,True
Kidney Beans,True,True,True,True
Milk,True,False,True,False
Nutmeg,True,True,False,False
Onion,True,True,False,False


## 演算法

### Apriori

#### **Support:**
Support is the frequency of A or how frequently an item appears in the dataset. It is defined as the fraction of the transaction T that contains the itemset X. If there are X datasets, then for transactions T, it can be written as:

<br>
<div align="center">

$Supp(X) = Freq(X) / T$

</div>
<br>


#### **Confidence:**
Confidence indicates how often the rule has been found to be true. Or how often the items X and Y occur together in the dataset when the occurrence of X is already given. It is the ratio of the transaction that contains X and Y to the number of records that contain X.

<br>
<div align="center">

$Confidence = Freq(X,Y) / Freq(X)$

</div>
<br>


#### **Lift:**
It is the strength of any rule, which can be defined as below formula: It is the ratio of the observed support measure and expected support if X and Y are independent of each other. It has three possible values:

<br>
<div align="center">

$Lift = Supp(X,Y) / Supp(X)*Supp(Y)$

</div>
<br>

- `Lift= 1`: The probability of occurrence of antecedent and consequent is independent of each other.
- `Lift>1`: It determines the degree to which the two itemsets are dependent to each other.
- `Lift<1`: It tells us that one item is a substitute for other items, which means one item has a negative effect on another.



<p align="right">
    <a href='https://www.geeksforgeeks.org/associative-classification-in-data-mining/'>
    source
    </a>
</p>

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

#### Step1: 跑演算法
這部分事前對 support 做篩選，當資料集大時可以透過提高 support 門檻加快速度。

In [ ]:
# 自己大概抓一下要設定多少
apriori(df_tran_matrix, min_support=0.5, use_colnames=True)

,support,itemsets
0,0.500,(Corn)
1,0.750,(Eggs)
2,0.500,(Ice cream)
3,0.625,(Kidney Beans)
4,0.500,(Onion)
5,0.500,"(Eggs, Kidney Beans)"


In [ ]:
# 設一個極小的門檻作為 min_support 可以列出所有結果
aprior_result = apriori(df_tran_matrix, min_support=0.00001, use_colnames=True)
aprior_result

,support,itemsets
0,0.125,(Apple)
1,0.500,(Corn)
2,0.125,(Dill)
3,0.750,(Eggs)
4,0.500,(Ice cream)
...,...,...
122,0.125,"(Kidney Beans, Eggs, Onion, Yogurt, Milk)"
123,0.125,"(Kidney Beans, Eggs, Onion, Yogurt, Nutmeg)"
124,0.125,"(Eggs, Onion, Yogurt, Milk, Nutmeg)"
125,0.125,"(Kidney Beans, Onion, Yogurt, Milk, Nutmeg)"


#### Step2: 計算評估指標

- antecedents 前因
- consequents 後果

In [ ]:
aprior_rules = association_rules(aprior_result)
aprior_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Apple),(Eggs),0.125,0.750,0.125,1.0,1.333333,0.031250,inf,0.285714
1,(Apple),(Kidney Beans),0.125,0.625,0.125,1.0,1.600000,0.046875,inf,0.428571
2,(Apple),(Milk),0.125,0.375,0.125,1.0,2.666667,0.078125,inf,0.714286
3,(Unicorn),(Corn),0.125,0.500,0.125,1.0,2.000000,0.062500,inf,0.571429
4,(Dill),(Eggs),0.125,0.750,0.125,1.0,1.333333,0.031250,inf,0.285714
...,...,...,...,...,...,...,...,...,...,...
400,"(Nutmeg, Kidney Beans)","(Yogurt, Milk, Eggs, Onion)",0.125,0.125,0.125,1.0,8.000000,0.109375,inf,1.000000
401,"(Yogurt, Onion)","(Milk, Nutmeg, Eggs, Kidney Beans)",0.125,0.125,0.125,1.0,8.000000,0.109375,inf,1.000000
402,"(Milk, Onion)","(Yogurt, Nutmeg, Eggs, Kidney Beans)",0.125,0.125,0.125,1.0,8.000000,0.109375,inf,1.000000
403,"(Yogurt, Nutmeg)","(Milk, Eggs, Kidney Beans, Onion)",0.125,0.125,0.125,1.0,8.000000,0.109375,inf,1.000000


#### Step3: 篩選出 pattern (pandas)


In [ ]:
aprior_rules[
    (aprior_rules['confidence'] > 0.9) &
    (aprior_rules['lift'] > 5)
    ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
33,"(Milk, Corn)",(Unicorn),0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,2
36,(Unicorn),"(Milk, Corn)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,1
39,"(Yogurt, Corn)",(Unicorn),0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,2
42,(Unicorn),"(Yogurt, Corn)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,1
109,"(Milk, Corn, Kidney Beans)",(Unicorn),0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...
400,"(Nutmeg, Kidney Beans)","(Yogurt, Milk, Eggs, Onion)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,2
401,"(Yogurt, Onion)","(Milk, Nutmeg, Eggs, Kidney Beans)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,2
402,"(Milk, Onion)","(Yogurt, Nutmeg, Eggs, Kidney Beans)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,2
403,"(Yogurt, Nutmeg)","(Milk, Eggs, Kidney Beans, Onion)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,2


In [ ]:
# 建立 關聯的 item 數
aprior_rules['antecedent_len'] = aprior_rules['antecedents'].str.len()

In [ ]:
aprior_rules[
    (aprior_rules['antecedent_len'] > 3) &
    (aprior_rules['confidence'] > 0.9) &
    (aprior_rules['lift'] > 5)
    ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
246,"(Yogurt, Milk, Corn, Kidney Beans)",(Unicorn),0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,4
369,"(Yogurt, Eggs, Kidney Beans, Onion)","(Milk, Nutmeg)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,4
370,"(Milk, Eggs, Kidney Beans, Onion)","(Yogurt, Nutmeg)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,4
373,"(Yogurt, Nutmeg, Eggs, Kidney Beans)","(Milk, Onion)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,4
374,"(Milk, Nutmeg, Eggs, Kidney Beans)","(Yogurt, Onion)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,4
379,"(Yogurt, Milk, Eggs, Onion)","(Nutmeg, Kidney Beans)",0.125,0.125,0.125,1.0,8.0,0.109375,inf,1.0,4


#### items-user
也可以用 items-user 去做

In [ ]:
# 改一下欄位名稱
df_item_matrix.columns = [f'user{i}' for i in df_item_matrix.columns]

In [ ]:
df_item_matrix

,user1,user2,user3,user4
items,,,,
Apple,True,False,False,False
Corn,True,True,True,True
Dill,False,True,False,False
Eggs,True,True,True,True
Ice cream,True,True,True,True
Kidney Beans,True,True,True,True
Milk,True,False,True,False
Nutmeg,True,True,False,False
Onion,True,True,False,False


In [ ]:
aprior_result_user = apriori(df_item_matrix, min_support=0.2, use_colnames=True)
aprior_result_user

,support,itemsets
0,0.818182,(user1)
1,0.636364,(user2)
2,0.636364,(user3)
3,0.363636,(user4)
4,0.545455,"(user2, user1)"
5,0.545455,"(user3, user1)"
6,0.363636,"(user4, user1)"
7,0.363636,"(user2, user3)"
8,0.363636,"(user2, user4)"
9,0.363636,"(user3, user4)"


In [ ]:
association_rules(aprior_result_user)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(user2),(user1),0.636364,0.818182,0.545455,0.857143,1.047619,0.024793,1.272727,0.125000
1,(user3),(user1),0.636364,0.818182,0.545455,0.857143,1.047619,0.024793,1.272727,0.125000
2,(user4),(user1),0.363636,0.818182,0.363636,1.000000,1.222222,0.066116,inf,0.285714
3,(user4),(user2),0.363636,0.636364,0.363636,1.000000,1.571429,0.132231,inf,0.571429
4,(user4),(user3),0.363636,0.636364,0.363636,1.000000,1.571429,0.132231,inf,0.571429
5,"(user2, user3)",(user1),0.363636,0.818182,0.363636,1.000000,1.222222,0.066116,inf,0.285714
6,"(user2, user4)",(user1),0.363636,0.818182,0.363636,1.000000,1.222222,0.066116,inf,0.285714
7,"(user4, user1)",(user2),0.363636,0.636364,0.363636,1.000000,1.571429,0.132231,inf,0.571429
8,(user4),"(user2, user1)",0.363636,0.545455,0.363636,1.000000,1.833333,0.165289,inf,0.714286
9,"(user3, user4)",(user1),0.363636,0.818182,0.363636,1.000000,1.222222,0.066116,inf,0.285714


### FP-Growth
加速版 aprior

In [ ]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [ ]:
fpgrowth_result = fpgrowth(df_tran_matrix, min_support=0.2, use_colnames=True)
fpgrowth_result

,support,itemsets
0,0.750,(Eggs)
1,0.625,(Kidney Beans)
2,0.500,(Onion)
3,0.375,(Yogurt)
4,0.375,(Milk)
5,0.250,(Nutmeg)
6,0.500,(Corn)
7,0.500,(Ice cream)
8,0.500,"(Eggs, Kidney Beans)"
9,0.375,"(Eggs, Onion)"


In [ ]:
association_rules(fpgrowth_result)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Kidney Beans),(Eggs),0.625,0.750,0.500,0.8,1.066667,0.031250,1.25,0.166667
1,"(Onion, Kidney Beans)",(Eggs),0.250,0.750,0.250,1.0,1.333333,0.062500,inf,0.333333
2,(Milk),(Kidney Beans),0.375,0.625,0.375,1.0,1.600000,0.140625,inf,0.600000
3,"(Yogurt, Milk)",(Kidney Beans),0.250,0.625,0.250,1.0,1.600000,0.093750,inf,0.500000
4,"(Yogurt, Kidney Beans)",(Milk),0.250,0.375,0.250,1.0,2.666667,0.156250,inf,0.833333
5,"(Milk, Eggs)",(Kidney Beans),0.250,0.625,0.250,1.0,1.600000,0.093750,inf,0.500000
6,(Nutmeg),(Onion),0.250,0.500,0.250,1.0,2.000000,0.125000,inf,0.666667
7,(Nutmeg),(Eggs),0.250,0.750,0.250,1.0,1.333333,0.062500,inf,0.333333
8,"(Nutmeg, Eggs)",(Onion),0.250,0.500,0.250,1.0,2.000000,0.125000,inf,0.666667
9,"(Nutmeg, Onion)",(Eggs),0.250,0.750,0.250,1.0,1.333333,0.062500,inf,0.333333


# reference
- [Associative Classification in Data Mining - GeeksforGeeks](https://www.geeksforgeeks.org/associative-classification-in-data-mining/)
- [Comparative Analysis Between Apriori and Fp Growth | Kaggle](https://www.kaggle.com/code/keitazoumana/comparative-analysis-between-apriori-and-fp-growth)
= [Apriori — Association Rule Mining In-depth Explanation and Python Implementation | by Chonyy | Towards Data Science](https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6)
- mlxtend doc
    - [Apriori - mlxtend (rasbt.github.io)](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
    - [Association rules - mlxtend (rasbt.github.io)](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)
    - [Fpgrowth - mlxtend (rasbt.github.io)](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/)